In [26]:
import numpy as np
import pandas as pd
import timecorr as tc
import hypertools as hyp
import supereeg as se
from matplotlib import pyplot as plt
from matplotlib import animation as ani
import seaborn as sns
import os
import glob as glob
from scipy.io import loadmat as load
import numba
import copy
import nilearn as nl
import nibabel as nib
from nilearn import plotting as ni_plt
from nilearn.input_data import NiftiMasker
from scipy.spatial.distance import pdist, cdist, squareform
from collections import deque
from IPython.display import HTML
import matlab.engine as mlab
import matlab
from neurosynth import Dataset
from neurosynth import meta, decode, network
import supereeg as se


from visbrain.objects import BrainObj, ColorbarObj, SceneObj, SourceObj
from visbrain.io import download_file, read_stc

%matplotlib inline

In [27]:
def largest_indices(ary, n):
    """Returns the n largest indices from a numpy array."""
    flat = ary.flatten()
    indices = np.argpartition(flat, -n)[-n:]
    indices = indices[np.argsort(-flat[indices])]
    return np.unravel_index(indices, ary.shape)

In [28]:
def smallest_indices(ary, n):
    """Returns the n largest indices from a numpy array."""
    flat = ary.flatten()
    indices = np.argpartition(flat, n)[:n]
    indices = indices[np.argsort(flat[indices])]
    return np.unravel_index(indices, ary.shape)

In [29]:
def rbf(centers, widths, locs):
    """
    Radial basis function
    Parameters
    ----------
    centers : ndarray
        rbf coordinates (one row per RBF)
    widths : ndarray
        RBF radii
    locs : ndarray
        locations to evaluate the RBFs (one row per location)
        
    Returns
    ----------
    results : ndarray
        Matrix of RBF weights for each RBF (row), at each location (column)
    """    
    weights = np.exp(np.divide(-cdist(locs, centers, metric='euclidean') ** 2, np.tile(np.array(widths, ndmin=2), [locs.shape[0], 1])))
    return weights.T

In [30]:
figdir = '../figs'
if not os.path.exists(figdir):
    os.mkdir(figdir)

In [31]:
neurosynth_dir ='../figs/neurosynth_data/'

In [32]:
n_f_dir = os.path.join(neurosynth_dir, 'figs')
if not os.path.exists(n_f_dir):
    os.mkdir(n_f_dir)

In [33]:
nii_dir = os.path.join(neurosynth_dir, 'niis')
if not os.path.exists(nii_dir):
    os.mkdir(nii_dir)

In [34]:
txt_dir = os.path.join(neurosynth_dir, 'txts')
if not os.path.exists(txt_dir):
    os.mkdir(txt_dir)

In [35]:
p_txt_dir = os.path.join(txt_dir, 'parsed_txts')
if not os.path.exists(p_txt_dir):
    os.mkdir(p_txt_dir)

In [36]:
ddir = '../../data/'

In [37]:
results_dir = os.path.join(os.getenv('HOME'), 'Desktop', 'timecorr_env', 'timecorr_paper', 'pieman', 'results')

In [38]:
posterior = load(os.path.join(results_dir, '../data/pieman_posterior_K700.mat'))
centers = posterior['posterior']['centers'][0][0][0][0][0]
widths = np.array(list(posterior['posterior']['widths'][0][0][0][0][0][:, 0].T))

In [39]:
data_dir = os.path.join(results_dir, 'mean_corrs')
corrs_dir = os.path.join(data_dir, 'corrs')

In [40]:
levels = np.arange(0,16,1)
conditions = ['intact', 'paragraph', 'rest', 'word']

In [41]:
template = se.helpers._gray(res=2)

## Try visbrain for plotting

In [32]:
top_n = 10

for l in [0,1,2,3,14]:
    for c in conditions:
        conds = glob.glob(os.path.join(data_dir, f'level_{l}', f'{c}.npy'))
        g_m = np.load(conds[0])

        networks = copy.copy(g_m)
        np.fill_diagonal(networks, 0)
        net_inds = largest_indices(np.triu(networks), top_n)
        pos_mask = networks[net_inds] > 0
        net_inds = np.concatenate((net_inds[0][pos_mask], net_inds[1][pos_mask]))
        temp_locs = centers[net_inds]
        temp_widths = widths[net_inds]
        
        w = rbf(temp_locs, temp_widths, template.get_locs().values)
        b = se.Brain(data=np.array(np.sum(w, axis=0), ndmin=2), locs=template.get_locs(), minimum_voxel_size=2)
        nii = se.Nifti(b)
        outfile = c+ '_' + str(l+1)
        nii.save(os.path.join(nii_dir, outfile + '_largest'))
        ni_plt.plot_glass_brain(nii, display_mode='lyrz', output_file=os.path.join(n_f_dir, outfile + '_largest.png'))

In [33]:
top_n = 10

for l in [0,1,2,3,14]:
    for c in conditions:
        conds = glob.glob(os.path.join(data_dir, f'level_{l}', f'{c}.npy'))
        g_m = np.load(conds[0])

        networks = copy.copy(g_m)
        np.fill_diagonal(networks, 0)
        net_inds = smallest_indices(np.triu(networks), top_n)
        neg_mask = networks[net_inds] < 0
        net_inds = np.concatenate((net_inds[0][neg_mask], net_inds[1][neg_mask]))
        temp_locs = centers[net_inds]
        temp_widths = widths[net_inds]
        
        w = rbf(temp_locs, temp_widths, template.get_locs().values)
        b = se.Brain(data=np.array(np.sum(w, axis=0), ndmin=2), locs=template.get_locs(), minimum_voxel_size=2)
        nii = se.Nifti(b)
        outfile = c+ '_' + str(l+1)
        nii.save(os.path.join(nii_dir, outfile + '_smallest'))
        ni_plt.plot_glass_brain(nii, display_mode='lyrz', output_file=os.path.join(n_f_dir, outfile + '_smallest.png'))

In [42]:
top_n = 10

for l in np.arange(0, 15, 1):
#for l in [0,1,2,3,14]:
    for c in conditions:
        conds = glob.glob(os.path.join(data_dir, f'level_{l}', f'{c}.npy'))
        g_m = np.load(conds[0])

        networks = copy.copy(g_m)
        np.fill_diagonal(networks, 0)
        net_inds = largest_indices(np.triu(np.abs(networks)), top_n)
        net_inds = np.concatenate((net_inds[0], net_inds[1]))
        temp_locs = centers[net_inds]
        temp_widths = widths[net_inds]
        
        w = rbf(temp_locs, temp_widths, template.get_locs().values)
        b = se.Brain(data=np.array(np.sum(w, axis=0), ndmin=2), locs=template.get_locs(), minimum_voxel_size=2)
        nii = se.Nifti(b)
        outfile = c+ '_' + str(l+1)
        nii.save(os.path.join(nii_dir, outfile + '_largest_abs'))
        ni_plt.plot_glass_brain(nii, display_mode='lyrz', output_file=os.path.join(n_f_dir, outfile + '_largest_abs.png'))

In [35]:
dataset = Dataset.load(os.path.join(neurosynth_dir, 'dataset.pkl'))
decoder = decode.Decoder(dataset=dataset, threshold=0.1, features=['taste', 'disgust', 'emotion', 'auditory', 'pain', 'somatosensory', 'conflict', 'switching', 'inhibition'])


In [43]:
import neurosynth as ns
import numpy as np
import pandas as pd
import timecorr as tc
import hypertools as hyp
from matplotlib import pyplot as plt
from matplotlib import animation as ani
import seaborn as sns
import os
import glob as glob
from scipy.io import loadmat as load
import numba
import copy
from scipy.spatial.distance import pdist, cdist, squareform
from collections import deque
from neurosynth import Dataset
from neurosynth import meta, decode, network
# import mkl
# mkl.set_num_threads(4)

#neurosynth_dir ='/dartfs/rc/lab/D/DBIC/CDL/f002s72/timecorr_paper/pieman/results/neurosynth_data'
nii_dir = os.path.join(neurosynth_dir, 'niis')
txt_dir = os.path.join(neurosynth_dir, 'txts')

# ns.dataset.download(path='neurosynth_dir', unpack=True)
# dataset = Dataset(os.path.join(neurosynth_dir,'database.txt'))
# dataset.add_features(os.path.join(neurosynth_dir, 'features.txt'))
# dataset.save(os.path.join(neurosynth_dir,'dataset.pkl'))

# dataset = Dataset.load(os.path.join(neurosynth_dir, 'dataset.pkl'))
# decoder = decode.Decoder(dataset=dataset, threshold=0.1, features=None)

conditions = ['intact', 'paragraph', 'rest', 'word']

for l in [1,2,3,4,15]:
    for c in conditions:
        in_file = c+ '_' + str(l)
        data = decoder.decode([os.path.join(nii_dir, in_file + '_smallest.nii')], save=None)
        renamed = data.reset_index()
        renamed.columns = ['feature', 'value']
        s_rename = renamed.sort_values(by=['value'], ascending=False)
        s_rename.to_csv(os.path.join(txt_dir, in_file +'_smallest.txt'))
        

for l in [1,2,3,4,15]:
    for c in conditions:
        in_file = c+ '_' + str(l)
        data = decoder.decode([os.path.join(nii_dir, in_file + '_largest.nii')], save=None)
        renamed = data.reset_index()
        renamed.columns = ['feature', 'value']
        s_rename = renamed.sort_values(by=['value'], ascending=False)
        s_rename.to_csv(os.path.join(txt_dir, in_file +'_largest.txt'))
        
for l in [1,2,3,4,15]:
    for c in conditions:
        in_file = c+ '_' + str(l)
        data = decoder.decode([os.path.join(nii_dir, in_file + '_largest_abs.nii')], save=None)
        renamed = data.reset_index()
        renamed.columns = ['feature', 'value']
        s_rename = renamed.sort_values(by=['value'], ascending=False)
        s_rename.to_csv(os.path.join(txt_dir, in_file +'_largest_abs.txt'))

In [20]:
conditions = ['intact', 'paragraph', 'rest', 'word']
full_pd = pd.DataFrame()
for l in [1,2,3,4]:
    for c in conditions:
        in_file = c+ '_' + str(l)
        temp_file = pd.read_csv(os.path.join(txt_dir, in_file +'_smallest.txt'))
        temp_file['condition'] = c
        temp_file['order'] = l
        if full_pd.empty:
                full_pd = temp_file[:10]
        else:
            full_pd = full_pd.append(temp_file[:10])

pretty_pd = full_pd[['order','condition', 'value', 'feature']]
pretty_pd.to_csv(os.path.join(txt_dir,'neg_10_features.csv'))

In [22]:
conditions = ['intact', 'paragraph', 'rest', 'word']
full_pd = pd.DataFrame()
for l in [1,2,3,4]:
    for c in conditions:
        in_file = c+ '_' + str(l)
        temp_file = pd.read_csv(os.path.join(txt_dir, in_file +'_largest_abs.txt'))
        temp_file['condition'] = c
        temp_file['order'] = l
        if full_pd.empty:
                full_pd = temp_file[:10]
        else:
            full_pd = full_pd.append(temp_file[:10])

pretty_pd = full_pd[['order','condition', 'value', 'feature']]
pretty_pd.to_csv(os.path.join(txt_dir,'abs_10_features.csv'))

In [23]:

conditions = ['intact', 'paragraph', 'rest', 'word']
full_pd = pd.DataFrame()
for l in [1,2,3,4]:
    for c in conditions:
        in_file = c+ '_' + str(l)
        temp_file = pd.read_csv(os.path.join(txt_dir, in_file +'_largest.txt'))
        temp_file['condition'] = c
        temp_file['order'] = l

        if full_pd.empty:
                full_pd = temp_file[:10]
        else:
            full_pd = full_pd.append(temp_file[:10])

pretty_pd = full_pd[['order','condition', 'value', 'feature']]
pretty_pd.to_csv(os.path.join(txt_dir,'pos_10_features.csv'))

In [151]:
pretty_pd = full_pd[['order','condition', 'value', 'feature']]

In [152]:
pretty_pd.to_csv(os.path.join(txt_dir,'negative_10_features.csv'))

In [177]:
### DO THIS ON THE CLUSTER for coactivateion maps
### only do this once
# import neurosynth as ns
# ns.dataset.download(path='neurosynth_dir', unpack=True)
# dataset = Dataset(os.path.join(neurosynth_dir,'database.txt'))
# dataset.add_features(os.path.join(neurosynth_dir, 'features.txt'))
# dataset.save(os.path.join(neurosynth_dir,'dataset.pkl'))

# dataset = Dataset.load(os.path.join(neurosynth_dir, 'dataset.pkl'))
# decoder = decode.Decoder(dataset=dataset, threshold=0.1, features=['taste', 'disgust', 'emotion', 'auditory', 'pain', 'somatosensory', 'conflict', 'switching', 'inhibition'])

# for e, label in enumerate(cond_labels):
#     order = label[0]
#     con = label[1]
#     top_10 = top_coors[:, :, e]

#     network.coactivation(dataset, top_10.tolist(), threshold=0.1, r=10, output_dir=neurosynth_dir, prefix=con+ '_' + str(order))
    
# for e, label in enumerate(cond_labels):
#     list_files = glob.glob(os.path.join(neurosynth_dir, con+ '_' + str(order + '*')))
#     decoder = decode.Decoder(dataset=dataset, threshold=0.1, features=['taste', 'disgust', 'emotion', 'auditory', 'pain', 'somatosensory', 'conflict', 'switching', 'inhibition'])  # can also pass other useful args
#     results_df = decoder.decode(images=list_files, save=None)  # can also pass names specific for the images
